In [113]:
# Importing necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from sklearn.datasets import fetch_openml
from sklearn.impute import SimpleImputer
import pandas as pd
# Loading the Boston housing dataset
# housing = fetch_openml(name="house_prices", as_frame=True)



In [114]:
data =  pd.read_csv('data.csv',encoding='utf-8')

In [115]:
data

,Name,Gender,Age,Spectacles,VA(OS),VA(OD),CDR(OS),CDR(OD),Diagnosis(OS),Diagnosis(OD),IOP (OS),IOP(OD)
0,????? ????,Male,64,0,0.60,0.70,0.5,0.9,0.0,1.0,14.0,23.0
1,???? ?????,Male,62,1,NaN,0.50,0.9,0.8,NaN,NaN,15.0,25.0
2,???? ????,Female,48,0,1.00,1.00,0.5,0.4,NaN,NaN,15.0,14.0
3,????? ??? ???,Male,60,0,0.50,0.20,0.7,0.7,1.0,1.0,30.0,30.0
4,???? ??? ????,Female,58,1,0.50,1.00,NaN,NaN,1.0,1.0,8.0,11.0
5,????? ???????,Female,65,1,0.40,1.00,0.9,0.8,1.0,0.0,25.0,10.0
6,????? ?????,Female,70,0,0.60,0.90,0.9,0.3,NaN,NaN,NaN,NaN
7,???? ??????,Male,50,0,0.10,0.90,0.8,0.7,0.0,0.0,16.0,16.0
8,??? ????,Female,25,1,0.10,0.70,0.4,0.8,NaN,NaN,7.0,9.0
9,????? ???,Female,64,0,0.50,0.70,0.3,0.9,1.0,0.0,30.0,NaN


In [116]:
# Create a new DataFrame to store the split rows
split_rows = []

# Iterate through each row of the original DataFrame
for index, row in data.iterrows():
    # Create a copy of the row for the left eye (OS)
    os_row = row.copy()
    os_row['Eye'] = 'OS'  # Left Eye
    os_row['VA'] = row['VA(OS)']
    os_row['CDR'] = row['CDR(OS)']
    os_row['Diagnosis'] = row['Diagnosis(OS)']
    os_row['IOP'] = row['IOP (OS)']
    split_rows.append(os_row)
    
    # Create a copy of the row for the right eye (OD)
    od_row = row.copy()
    od_row['Eye'] = 'OD'  # Right Eye
    od_row['VA'] = row['VA(OD)']
    od_row['CDR'] = row['CDR(OD)']
    od_row['Diagnosis'] = row['Diagnosis(OD)']
    od_row['IOP'] = row['IOP(OD)']
    split_rows.append(od_row)

# Create a new DataFrame from the split rows
split_df = pd.DataFrame(split_rows)

# Drop the unnecessary columns
split_df.drop(columns=['VA(OS)', 'VA(OD)', 'CDR(OS)', 'CDR(OD)', 'Diagnosis(OS)', 'Diagnosis(OD)', 'IOP (OS)', 'IOP(OD)'], inplace=True)

# Display the new DataFrame
print(split_df)


            Name  Gender  Age  Spectacles Eye   VA  CDR  Diagnosis   IOP
0     ????? ????    Male   64           0  OS  0.6  0.5        0.0  14.0
0     ????? ????    Male   64           0  OD  0.7  0.9        1.0  23.0
1     ???? ?????    Male   62           1  OS  NaN  0.9        NaN  15.0
1     ???? ?????    Male   62           1  OD  0.5  0.8        NaN  25.0
2      ???? ????  Female   48           0  OS  1.0  0.5        NaN  15.0
..           ...     ...  ...         ...  ..  ...  ...        ...   ...
42  ????? ??????  Female   55           1  OD  NaN  1.0        0.0   NaN
43     ???? ????    Male   71           0  OS  0.3  0.5        1.0  12.0
43     ???? ????    Male   71           0  OD  0.6  0.7        1.0  13.0
44     ???? ????    Male   65           1  OS  0.5  0.3        NaN  15.0
44     ???? ????    Male   65           1  OD  0.2  0.3        NaN  12.0

[90 rows x 9 columns]


In [117]:
split_df

,Name,Gender,Age,Spectacles,Eye,VA,CDR,Diagnosis,IOP
0,????? ????,Male,64,0,OS,0.6,0.5,0.0,14.0
0,????? ????,Male,64,0,OD,0.7,0.9,1.0,23.0
1,???? ?????,Male,62,1,OS,NaN,0.9,NaN,15.0
1,???? ?????,Male,62,1,OD,0.5,0.8,NaN,25.0
2,???? ????,Female,48,0,OS,1.0,0.5,NaN,15.0
...,...,...,...,...,...,...,...,...,...
42,????? ??????,Female,55,1,OD,NaN,1.0,0.0,NaN
43,???? ????,Male,71,0,OS,0.3,0.5,1.0,12.0
43,???? ????,Male,71,0,OD,0.6,0.7,1.0,13.0
44,???? ????,Male,65,1,OS,0.5,0.3,NaN,15.0


In [118]:
# Set Diagnosis to 1 where CDR > 0.5 and Diagnosis is NaN
split_df.loc[(split_df['CDR'] > 0.5) & (split_df['Diagnosis'].isnull()), 'Diagnosis'] = 1

# Set Diagnosis to 0 where CDR <= 0.5 and Diagnosis is NaN
split_df.loc[((split_df['CDR'] <= 0.5) | (split_df['CDR'].isnull())) & (split_df['Diagnosis'].isnull()), 'Diagnosis'] = 0


In [119]:
gender_mapping = {'Male': 0, 'Female': 1}
split_df['Gender'] = split_df['Gender'].map(gender_mapping)

eye_mapping = {'OS': 0, 'OD': 1}
split_df['Eye'] = split_df['Eye'].map(eye_mapping)


In [120]:
split_df

,Name,Gender,Age,Spectacles,Eye,VA,CDR,Diagnosis,IOP
0,????? ????,0,64,0,0,0.6,0.5,0.0,14.0
0,????? ????,0,64,0,1,0.7,0.9,1.0,23.0
1,???? ?????,0,62,1,0,NaN,0.9,1.0,15.0
1,???? ?????,0,62,1,1,0.5,0.8,1.0,25.0
2,???? ????,1,48,0,0,1.0,0.5,0.0,15.0
...,...,...,...,...,...,...,...,...,...
42,????? ??????,1,55,1,1,NaN,1.0,0.0,NaN
43,???? ????,0,71,0,0,0.3,0.5,1.0,12.0
43,???? ????,0,71,0,1,0.6,0.7,1.0,13.0
44,???? ????,0,65,1,0,0.5,0.3,0.0,15.0


In [121]:
split_df.drop('Name', axis=1, inplace=True)

# Create a new "ID" column with sequential numbers starting from 1
split_df['ID'] = range(1, len(split_df) + 1)

In [122]:
X_data = split_df.drop('Diagnosis',axis=1)
y_data = split_df['Diagnosis']

In [123]:
y_data

0     0.0
0     1.0
1     1.0
1     1.0
2     0.0
     ... 
42    0.0
43    1.0
43    1.0
44    0.0
44    0.0
Name: Diagnosis, Length: 90, dtype: float64

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.1, random_state=44, shuffle=True)


In [163]:
imputer = SimpleImputer(strategy='mean')
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)


In [164]:
KNeighborsRegressorModel = KNeighborsRegressor(n_neighbors=3, weights='uniform', algorithm='auto')
KNeighborsRegressorModel.fit(X_train_imputed, y_train)

KNeighborsRegressor(n_neighbors=3)

In [165]:
print('KNeighborsRegressorModel Train Score is : ', KNeighborsRegressorModel.score(X_train_imputed, y_train))
print('KNeighborsRegressorModel Test Score is : ', KNeighborsRegressorModel.score(X_test_imputed, y_test))


KNeighborsRegressorModel Train Score is :  0.416156670746634
KNeighborsRegressorModel Test Score is :  -0.9285714285714286


In [166]:
y_pred = KNeighborsRegressorModel.predict(X_test_imputed)
print('Predicted Value for KNeighborsRegressorModel is : ', y_pred[:10])

# Calculating mean absolute error
MAEValue = mean_absolute_error(y_test, y_pred, multioutput='uniform_average')
print('Mean Absolute Error Value is : ', MAEValue)

# Calculating mean squared error
MSEValue = mean_squared_error(y_test, y_pred, multioutput='uniform_average')
print('Mean Squared Error Value is : ', MSEValue)

# Calculating median squared error
MdSEValue = median_absolute_error(y_test, y_pred)
print('Median Squared Error Value is : ', MdSEValue)

Predicted Value for KNeighborsRegressorModel is :  [0.66666667 0.33333333 0.33333333 0.66666667 1.         1.
 0.66666667 1.         0.66666667]
Mean Absolute Error Value is :  0.48148148148148145
Mean Squared Error Value is :  0.3333333333333333
Median Squared Error Value is :  0.6666666666666666
